In [ ]:
import os
import re
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the Dataset class
class BMIDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.image_files = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        match = re.match(r"\d+?_([FMfm])_(\d+?)_(\d+?)_(\d+).+", img_name)
        if match:
            height = int(match.group(3)) / 100000  # convert mm to m
            weight = int(match.group(4)) / 100000  # convert mg to kg
            bmi = weight / (height ** 2)
        else:
            bmi = 0.0
        return image, float(bmi)

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Set the folder paths
train_folder = "/content/drive/MyDrive/Image_train/"
val_folder = "/content/drive/MyDrive/Image_val/"
# train_folder = "/content/drive/MyDrive/Image_train_gaussian_15/"
# val_folder = "/content/drive/MyDrive/Image_val_gaussian_15/"
# train_folder = "/content/drive/MyDrive/Image_train_gaussian_30/"
# val_folder = "/content/drive/MyDrive/Image_val_gaussian_30/"
# train_folder = "/content/drive/MyDrive/Image_train_rotate_10/"
# val_folder = "/content/drive/MyDrive/Image_val_rotate_10/"
# train_folder = "/content/drive/MyDrive/Image_train_rotate_20/"
# val_folder = "/content/drive/MyDrive/Image_val_rotate_20/"

# Create datasets and dataloaders
train_dataset = BMIDataset(train_folder, transform=transform)
val_dataset = BMIDataset(val_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

# Build the ResNet101 model for regression
model = models.resnet101(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)
model = model.to(DEVICE)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)



In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.to(DEVICE)
        targets = targets.to(DEVICE).unsqueeze(1).float()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Evaluation on validation set
model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for images, targets in val_loader:
        images = images.to(DEVICE)
        targets = targets.to(DEVICE).unsqueeze(1).float()
        outputs = model(images)
        all_preds.extend(outputs.cpu().numpy().flatten().tolist())
        all_targets.extend(targets.cpu().numpy().flatten().tolist())

mae_val = mean_absolute_error(all_targets, all_preds)
print(f"Validation MAE: {mae_val:.4f}")
